In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
!pip3 install tensorflow==1.4
import tensorflow as tf

from commodities import hard

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 39.3 MB 90 kB/s  eta 0:00:01     |██████████████████████████▍     | 32.5 MB 562 kB/s eta 0:00:13
     |████████████████████████████████| 1.7 MB 811 kB/s eta 0:00:01
     |████████████████████████████████| 889 kB 1.2 MB/s eta 0:00:01
  Created wheel for html5lib: filename=html5lib-0.9999999-py3-none-any.whl size=107220 sha256=2c4eb4c97821480f8c176cf6259444aaf73b6d6490ec554f197fd9f804e8b193
  Stored in directory: /Users/jamesbaker/Library/Caches/pip/wheels/90/1c/cb/a87fd097ff74648ecc468a703001f6c7c86d8a71d459e65c98
Successfully built html5lib
  Attempting uninstall: bleach
    Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorfl

/Users/jamesbaker/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jamesbaker/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jamesbaker/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jamesbaker/miniconda3/lib/python3.6/site-packages/tensorf

In [31]:
from collections import deque
import random

name = 'Q-learning agent'

class Agent:
    def __init__(self, state_size, window_size, trend, skip, batch_size):
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        self.action_size = 3
        self.batch_size = batch_size
        self.memory = deque(maxlen = 1000)
        self.inventory = []
        self.performance=[]
        self.gamma = 0.95
        self.epsilon = 0.5
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999

        tf.reset_default_graph()
        self.sess = tf.InteractiveSession()
        self.X = tf.placeholder(tf.float32, [None, self.state_size])
        self.Y = tf.placeholder(tf.float32, [None, self.action_size])
        feed = tf.layers.dense(self.X, 256, activation = tf.nn.relu)
        self.logits = tf.layers.dense(feed, self.action_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.GradientDescentOptimizer(1e-5).minimize(
            self.cost
        )
        self.sess.run(tf.global_variables_initializer())

    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        return np.argmax(
            self.sess.run(self.logits, feed_dict = {self.X: state})[0]
        )
    
    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array([res])

    def replay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size, l):
            mini_batch.append(self.memory[i])
        replay_size = len(mini_batch)
        X = np.empty((replay_size, self.state_size))
        Y = np.empty((replay_size, self.action_size))
        states = np.array([a[0][0] for a in mini_batch])
        new_states = np.array([a[3][0] for a in mini_batch])
        Q = self.sess.run(self.logits, feed_dict = {self.X: states})
        Q_new = self.sess.run(self.logits, feed_dict = {self.X: new_states})
        for i in range(len(mini_batch)):
            state, action, reward, next_state, done = mini_batch[i]
            target = Q[i]
            target[action] = reward
            if not done:
                target[action] += self.gamma * np.amax(Q_new[i])
            X[i] = state
            Y[i] = target
        cost, _ = self.sess.run(
            [self.cost, self.optimizer], feed_dict = {self.X: X, self.Y: Y}
        )
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return cost
    
    def buy(self, initial_money):
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        state = self.get_state(0)
        for t in range(0, len(self.trend) - 1, self.skip):
            self.performance.append(initial_money)
            action = self.act(state)
            next_state = self.get_state(t + 1)
            
            if action == 1 and initial_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                inventory.append(self.trend[t])
                initial_money -= self.trend[t]
                states_buy.append(t)
                #print('day %d: buy 1 unit at price %f, total balance %f'% (t, self.trend[t], initial_money))
                
                
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.trend[t]
                states_sell.append(t)
                try:
                    invest = ((close[t] - bought_price) / bought_price) * 100
                except:
                    invest = 0
                '''print(
                    'day %d, sell 1 unit at price %f, investment %f %%, total balance %f,'
                    % (t, close[t], invest, initial_money)
                )'''
            
            state = next_state
        self.performance.append(initial_money)
        invest = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        return states_buy, states_sell, total_gains, invest
        
    def train(self, iterations, checkpoint, initial_money):
        for i in range(iterations):
            total_profit = 0
            inventory = []
            state = self.get_state(0)
            starting_money = initial_money
            for t in range(0, len(self.trend) - 1, self.skip):
                action = self.act(state)
                next_state = self.get_state(t + 1)
                
                if action == 1 and starting_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                    inventory.append(self.trend[t])
                    starting_money -= self.trend[t]
                
                elif action == 2 and len(inventory) > 0:
                    bought_price = inventory.pop(0)
                    total_profit += self.trend[t] - bought_price
                    starting_money += self.trend[t]
                    
                invest = ((starting_money - initial_money) / initial_money)
                self.memory.append((state, action, invest, 
                                    next_state, starting_money < initial_money))
                state = next_state
                batch_size = min(self.batch_size, len(self.memory))
                cost = self.replay(batch_size)
            if (i+1) % checkpoint == 0:
                print('epoch: %d, total rewards: %f., cost: %f, total money: %f'%(i + 1, total_profit, cost,
                                                                                  starting_money))

In [3]:
hard

,value_gold,value_oil,value_silver,value_ruth,value_irid,value_pall,value_rhod,value_plat
Date,,,,,,,,
1992-07-01,343.400,21.90,4.045,32.0,200.0,83.0,2675.0,385.0
1992-07-02,344.300,22.08,4.050,32.0,200.0,84.0,2725.0,389.0
1992-07-03,346.130,22.08,4.060,32.0,200.0,84.0,2725.0,386.0
1992-07-06,346.250,21.87,4.060,32.0,200.0,84.0,2725.0,388.0
1992-07-07,346.550,21.51,4.060,32.0,200.0,83.0,2700.0,383.0
...,...,...,...,...,...,...,...,...
2020-04-21,1692.195,8.91,15.255,270.0,1575.0,1886.0,8800.0,732.0
2020-04-22,1690.465,13.64,14.860,270.0,1575.0,1971.0,8100.0,760.0
2020-04-23,1717.410,15.06,15.245,270.0,1575.0,2014.0,8200.0,770.0


In [8]:
asset='value_gold'
start='2015-01-01'
hard[asset].loc[start:].index

DatetimeIndex(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13',
               '2015-01-14', '2015-01-15',
               ...
               '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17',
               '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23',
               '2020-04-24', '2020-04-27'],
              dtype='datetime64[ns]', name='Date', length=1259, freq=None)

In [34]:
start='2015-01-01'
perf={}
agents={}
for asset in hard.columns:
    print(asset)
    close = hard[asset].loc[:start].tolist()
    initial_money = 100000
    window_size = 30
    skip = 1
    batch_size = 32
    agent = Agent(state_size = window_size, 
              window_size = window_size, 
              trend = close, 
              skip = skip, 
              batch_size = batch_size)
    agent.train(iterations = 100, checkpoint = 25, initial_money = initial_money)
    agent.trend=hard[asset].loc[start:].tolist()
    buy=agent.buy(initial_money)
    print('returns: {} %'.format(buy[3]))
    perf[asset]=pd.Series(agent.performance,index=hard[asset].loc[start:].index)
    agents[asset]=agent
    

value_gold
epoch: 25, total rewards: 267791.970000., cost: 0.286122, total money: 3995.170000
epoch: 50, total rewards: 261898.915000., cost: 0.114100, total money: 3385.635000
epoch: 75, total rewards: 232304.180000., cost: 0.090121, total money: 2940.605000
epoch: 100, total rewards: 232177.905000., cost: 0.028061, total money: 4045.185000
returns: -71.90572499999992 %
value_oil
epoch: 25, total rewards: 14.450000., cost: 0.071556, total money: 100014.450000
epoch: 50, total rewards: 15.660000., cost: 0.056023, total money: 100015.660000
epoch: 75, total rewards: 64.590000., cost: 0.080992, total money: 100064.590000
epoch: 100, total rewards: -12.240000., cost: 0.026748, total money: 99987.760000
returns: -0.06055000000004657 %
value_silver
epoch: 25, total rewards: -56.675000., cost: 0.008402, total money: 99943.325000
epoch: 50, total rewards: 17.060000., cost: 0.005302, total money: 100017.060000
epoch: 75, total rewards: 16.605000., cost: 0.004257, total money: 100016.605000
epo

In [37]:
from testing import test_model_mapping,series_merge
from trading import generate_signals, eval_signals,time_series_chart

In [57]:
df=pd.DataFrame()
for k,v in perf.items():
    df[k]=v/100000

In [58]:
df

,value_gold,value_oil,value_silver,value_ruth,value_irid,value_pall,value_rhod,value_plat
Date,,,,,,,,
2015-01-02,1.000000,1.000000,1.000000,1.00000,1.0000,1.00000,1.00000,1.00000
2015-01-05,1.000000,0.999473,0.999841,1.00000,1.0000,1.00000,1.00000,1.00000
2015-01-06,1.000000,0.999473,1.000001,1.00000,1.0000,0.99202,1.00000,0.98790
2015-01-07,0.987924,0.999953,1.000001,0.99944,1.0000,0.98401,1.00000,1.00006
2015-01-08,0.987924,0.999953,1.000001,1.00000,1.0000,0.99200,1.00000,0.98785
...,...,...,...,...,...,...,...,...
2020-04-21,0.246847,0.999394,1.000242,1.02845,1.1259,0.19095,1.12815,0.99692
2020-04-22,0.263769,0.999394,1.000242,1.03115,1.1259,0.19095,1.12815,0.99692
2020-04-23,0.263769,0.999394,1.000242,1.03115,1.1259,0.19095,1.12815,0.99692


In [59]:
df.to_csv('q_results.csv',index=True)